# Make 300-frame center clips (using `hybrid.io.tif` reader)
Ноутбук проходит по выбранным TIFF-файлам и сохраняет клипы по 300 кадров из середины.
Используется **наш ридер** из `hybrid.io.tif` (`read_tiff_memmap`/`read_tiff_stack`).

**Отредактируйте параметры в первой ячейке.**

In [5]:
# ---- Config ----
from pathlib import Path
import sys

# Путь к корню проекта (если ноутбук в .../notebooks, то корень — это '..')
PROJECT_ROOT = Path('..').resolve()
DATA_DIR = (PROJECT_ROOT / 'notebooks' / 'data').resolve()   # поменяйте при необходимости
OUT_DIR = (DATA_DIR / 'shorts').resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Добавляем src в PYTHONPATH, чтобы импортировать hybrid.* если пакет не установлен
SRC_DIR = (PROJECT_ROOT / 'src')
if SRC_DIR.exists():
    sys.path.insert(0, str(SRC_DIR))

# Вырезаем ровно столько кадров
N_FRAMES = 300

# Где брать окно: 'center' | 'start' | 'end'
WINDOW_POS = 'center'

# Явный список файлов (basename). Если пусто — берём все *.tif/*.tiff в DATA_DIR
SELECT: List[str] = ['MoS1b_s1_C1_M1_T1_CLB_2025_04_08.tiff',
                     'MoS1b_s1_C1_M1_T1_FL2_2025_04_08.tiff',
                     'MoS1b_s1_C1_M1_T1_TRE_2025_04_08.tiff',
                     'MoS1b_s2_C1_M2_T2_FL2_2025_04_08.tiff',
                     'MoS1b_s2_C1_M2_T2_TRE_2025_04_08.tiff',
                     'MoS1b_s3_C1_M3_T3_CLB_2025_04_08.tiff',
                     'MoS1b_s3_C1_M3_T3_FL2_2025_04_08.tiff',
                     'MoS1b_s3_C1_M3_T3_TRE_2025_04_08.tiff',
                     'MoS1b_s4_C2_M5_T2_CLB_2025_04_08.tiff',
                     'MoS1b_s4_C2_M5_T2_FL2_2025_04_08.tiff',
                     'MoS1b_s4_C2_M5_T2_TRE_2025_04_08.tiff']

In [6]:
# ---- Imports (project reader) ----
from typing import List, Tuple
import numpy as np
import tifffile as tiff

# наш ридер
from hybrid.io.tif import read_tiff_memmap, read_tiff_stack

In [7]:
# ---- Helpers ----
def list_candidate_files(data_dir: Path, select: list) -> list:
    if select:
        paths = [ (data_dir / s) for s in select ]
    else:
        paths = sorted(list(data_dir.glob('*.tif')) + list(data_dir.glob('*.tiff')))
    paths = [p for p in paths if p.exists() and p.is_file()]
    return paths

def choose_window(n_total: int, n_take: int, where: str='center'):
    n_take = min(n_take, n_total)
    if where == 'start':
        start = 0
    elif where == 'end':
        start = max(0, n_total - n_take)
    else:
        start = max(0, (n_total - n_take)//2)
    end = start + n_take
    return start, end

def save_tiff_stack(out_path: Path, stack: np.ndarray, imwrite_kw: dict | None = None):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with tiff.TiffWriter(str(out_path), bigtiff=True) as tw:
        for i in range(stack.shape[0]):
            tw.write(stack[i], **(imwrite_kw or {}))

In [8]:
# ---- Run ----
paths = list_candidate_files(DATA_DIR, SELECT)
print(f"Found {len(paths)} candidate file(s) in {DATA_DIR}")

for p in paths:
    try:
        # Сначала пытаемся memmap (без полной загрузки в RAM)
        try:
            movie = read_tiff_memmap(str(p))  # ожидаем (T, H, W)
        except Exception as e_mem:
            # если файл не мэпится (компрессия и пр.) — fallback на полный стек
            movie = read_tiff_stack(str(p), normalize=False)
        
        if movie.ndim != 3:
            raise ValueError(f"Unexpected shape {movie.shape}; need 3D (T, H, W)")
        
        T = movie.shape[0]
        if T < 2:
            print(f"[skip] {p.name}: not a stack (T={T})")
            continue

        start, end = choose_window(T, N_FRAMES, WINDOW_POS)
        clip = np.asarray(movie[start:end])

        out_name = p.stem + f"_clip_{start:06d}-{end:06d}.tiff"
        out_path = OUT_DIR / out_name
        save_tiff_stack(out_path, clip, imwrite_kw=dict(compression=None))
        print(f"[ok] {p.name} -> {out_path.name}  ({clip.shape[0]} frames)")
    except Exception as e:
        print(f"[ERROR] {p.name}: {e}")

Found 11 candidate file(s) in D:\Projects\hybrid\notebooks\data
[I/O] Using full-RAM imread (~2.23 GiB): D:\Projects\hybrid\notebooks\data\MoS1b_s1_C1_M1_T1_CLB_2025_04_08.tiff
[ok] MoS1b_s1_C1_M1_T1_CLB_2025_04_08.tiff -> MoS1b_s1_C1_M1_T1_CLB_2025_04_08_clip_000825-001125.tiff  (300 frames)
[I/O] Using full-RAM imread (~2.23 GiB): D:\Projects\hybrid\notebooks\data\MoS1b_s1_C1_M1_T1_FL2_2025_04_08.tiff
[ok] MoS1b_s1_C1_M1_T1_FL2_2025_04_08.tiff -> MoS1b_s1_C1_M1_T1_FL2_2025_04_08_clip_000825-001125.tiff  (300 frames)
[I/O] Using full-RAM imread (~2.23 GiB): D:\Projects\hybrid\notebooks\data\MoS1b_s1_C1_M1_T1_TRE_2025_04_08.tiff
[ok] MoS1b_s1_C1_M1_T1_TRE_2025_04_08.tiff -> MoS1b_s1_C1_M1_T1_TRE_2025_04_08_clip_000825-001125.tiff  (300 frames)
[I/O] Using full-RAM imread (~2.23 GiB): D:\Projects\hybrid\notebooks\data\MoS1b_s2_C1_M2_T2_FL2_2025_04_08.tiff
[ok] MoS1b_s2_C1_M2_T2_FL2_2025_04_08.tiff -> MoS1b_s2_C1_M2_T2_FL2_2025_04_08_clip_000825-001125.tiff  (300 frames)
[I/O] Using full